# Turn water observations into waterbody polygons

* **Products used:** 
[wofs_ls_summary_alltime](https://explorer.digitalearth.africa/products/wofs_ls_summary_alltime)
* **Special requirements:** 
This notebook requires the [python_geohash](https://pypi.org/project/python-geohash/) library. You can install it locally by using `python -m pip install python-geohash`.
* **Prerequisites:** 
    * A coastline polygon to filter out polygons generated from ocean pixels.
        * Variable name: `land_sea_mask_fp`
        * Here we have used the [Marine Regions Global Oceans and Seas v01 dataset](https://www.marineregions.org/sources.php#goas).
* **Optional prerequisites:**
    * River line dataset for filtering out polygons comprised of river segments.
        * Variable name: `major_rivers_fp`
        * The option to filter out major rivers is provided, and so this dataset is optional if `filter_out_rivers = False`.
        * We therefore turn this off during the production of the water bodies shapefile. 
    * Urban high rise polygon dataset
        * Variable name: `urban_mask_fp`, but this is optional and can be skipped by setting `filter_out_urban_areas = False`.
        * WOfS has a known limitation, where deep shadows thrown by tall CBD buildings are misclassified as water. This results in 'waterbodies' around these misclassified shadows in capital cities. If you are not using WOfS for your analysis, you may choose to set `filter_out_urban_areas = False`.
        * Here we haved generated a polygon dataset to act as our `urban_mask` by thresholding the High Resolution Population Density Maps dataset.

## Background

Water is among one the most precious natural resources and is essential for the survival of life on Earth. For many countries in Africa, the scarcity of water is both an economic and social issue. Water is required not only for consumption but for industries and environmental ecosystems to function and flourish. 

With the demand for water increasing, there is a need to better understand our water availability to ensure we are managing our water resources effectively and efficiently.  

Digital Earth Africa (DE Africa)'s [Water Observations from Space (WOfS) dataset](https://docs.digitalearthafrica.org/en/latest/data_specs/Landsat_WOfS_specs.html), provides a water classified image of Africa approximately every 16 days. These individual water observations have been combined into a [WOfS All-Time Summary](https://explorer.digitalearth.africa/products/wofs_ls_summary_alltime) product, which calculates the frequency of wet observations (compared against all clear observations of that pixel), over the full 30-plus years satellite archive. 

The WOfS All-Time Summary product provides valuable insights into the persistence of water across the African landscape on a pixel by pixel basis. While knowing the wet history of a single pixel within a waterbody is useful, it is more useful to be able to map the whole waterbody as a single object. 

This notebook demonstrates a workflow for mapping waterbodies across Africa as polygon objects. This workflow has been used to produce **DE Africa Waterbodies**. 

## Description
This code follows the following workflow:

* Load the required python packages
* Load the required functions
* Set your chosen analysis parameters:
    * set up some file names for the inputs and outputs
    * create a datacube query object
    * set the analysis region
    * wetness threshold/s
    * min/max waterbody size
    * minimum number of valid observations
    * read in a land/sea mask
    * optional flag to filter out waterbodies that intersect with major rivers
        * if you set this flag you will need to provide a dataset to do the filtering
    * read in an urban mask
* Generate the first temporary polygon set:
  * For each tile:
    * Load the WOfS All Time Summary Dataset
    * Keep only pixels observed at least x times
    * Keep only pixels identified as wet at least x% of the time
        * Here the code can take in two wetness thresholds, to produce two initial temporary polygon files.
    * Convert the raster data into polygons
    * Append the polygon set to a temporary shapefile
* Remove artificial polygon borders created at tile boundaries by merging polygons that intersect across tile boundaries
* Filter the combined polygon dataset (note that this step happens after the merging of tile boundary polygons to ensure that artifacts are not created by part of a polygon being filtered out, while the remainder of the polygon that sits on a separate tile is treated differently).
    * Filter the polygons based on area / size
    * Remove polygons that intersect with Africa's coastline
    * Remove erroneous 'water' polygons within high-rise CBD areas
    * Combine the two generated wetness thresholds (optional)
    * Optional filtering for proximity to major rivers  
* Save out the final polygon set to a shapefile

## Load python packages

In [ ]:
# Uncomment the line below to install the python_geohash package.
# !python -m pip install python_geohash

In [ ]:
import os
import math
import shutil  #
import pyproj
import fiona
import shapely
import datetime
import rioxarray
import numpy as np
import pandas as pd
import geopandas as gpd
from pathlib import Path
import geohash as gh

import datacube
from datacube.utils import geometry

from deafrica_tools.areaofinterest import define_area
from deafrica_tools.spatial import xr_vectorize, xr_rasterize

## Define functions required in the script

In [ ]:
def get_tiles(aoi_geopolygon=None, all_of_africa=True, crs="EPSG:4326"):
    """
    Returns a GeoDataFrame of the tiles intersecting with the area of 
    interest Geometry object or all tiles/regions from the WOfS 
    All time summary product.

    Parameters
    ----------
    aoi_geopolygon: datacube.utils.geometry._base.Geometry
        Geometry object of the area of interest
    all_of_africa : bool
        Default True. If set to True, all the tiles present in the 
        WOfS All time summary product regions GeoJSON file will be loaded.
        If set to True aoi_geopolygon must be set to None. 
    crs: str
        CRS of the returned tiles GeoDataFrame. Defaults to "EPSG:4326"
    
    Returns
    -------
    geopandas.geodataframe.GeoDataFrame

    """
    # Set important variables.
    wofs_ls_summary_alltime_regions_url = "https://explorer.digitalearth.africa/api/regions/wofs_ls_summary_alltime"

    # Convert the area of interest geopolygon to the specified crs.
    aoi_geopolygon = aoi_geopolygon.to_crs(crs)

    # Checks on arguments passed to function.
    if not all_of_africa and aoi_geopolygon is None:
        raise Exception(
            "Please pass a Geometry object to aoi_geopolygon if all_of_africa is set to False."
        )
    elif all_of_africa and aoi_geopolygon is not None:
        raise Exception(
            "Please pass None to aoi_geopolygon if all_of_africa is set to True."
        )

    if all_of_africa and aoi_geopolygon is None:
        tiles = gpd.read_file(wofs_ls_summary_alltime_regions_url).drop(
            'count', axis=1).to_crs(crs)
    elif not all_of_africa and aoi_geopolygon is not None:
        africa_tiles = gpd.read_file(wofs_ls_summary_alltime_regions_url).drop(
            'count', axis=1).to_crs(crs)

        keep_tiles_list = []
        for row in africa_tiles.itertuples():
            row_geopolygon = geometry.Geometry(geom=row.geometry, crs=crs)
            if geometry.intersects(row_geopolygon, aoi_geopolygon):
                keep_tiles_list.append(row.region_code)

        tiles = africa_tiles[africa_tiles['region_code'].isin(
            keep_tiles_list)].reset_index(drop=True)

    return tiles

In [ ]:
def filter_shapefile_by_intersection(gpd_data,
                                     gpd_filter,
                                     filtertype="intersects",
                                     invert_mask=True,
                                     return_inverse=False):
    """
    Filter out polygons that intersect with another polygon shapefile. 
    
    Parameters
    ----------
    
    gpd_data: geopandas GeoDataFrame
        Polygon data to be filtered.
    gpd_filter: geopandas GeoDataFrame
        Polygon dataset to be used as a filter.
    
    Optional
    --------
    filtertype: default = 'intersects'
        Options = ['intersects', 'contains', 'within']
    invert_mask: boolean
        Default = 'True'. This determines whether you want areas that 
        DO ( = 'False') or DON'T ( = 'True') intersect with the filter dataset.
    return_inverse: boolean
        Default = 'False'. If True, then return both parts of the intersection:
        - those that intersect AND 
        - those that don't as two GeoDataFrames.
    
    Returns
    -------
    gpd_data_filtered: geopandas GeoDataFrame
        If invert_mask==True, `gpd_data_filtered` is a filtered polygon set, 
        with polygons that DO intersect with gpd_filter removed. 
        If invert_mask==False, `gpd_data_filtered` is a filtered polygon set, 
        with polygons that DON'T intersect with gpd_filter removed. 
    intersect_index: list of indices of gpd_data that intersect with gpd_filter.
    
    Optional
    --------
    if 'return_inverse = True'
    gpd_data_inverse: geopandas GeoDataFrame
        If invert_mask==True, `gpd_data_inverse` is a filtered polygon set, 
        with polygons that DON'T intersect with gpd_filter removed (inverse of gpd_data_filtered).
        If invert_mask==False, `gpd_data_inverse` is a filtered polygon set, 
        with polygons that DO intersect with gpd_filter removed (inverse of gpd_data_filtered). 
        
    """

    # Check that the coordinate reference systems of both GeoDataFrames are the same.
    assert gpd_data.crs == gpd_filter.crs

    # Find the index of all the polygons in gpd_data that intersect with gpd_filter.
    intersections = gpd_filter.sjoin(gpd_data,
                                     how="inner",
                                     predicate=filtertype)
    intersect_index = np.sort(intersections["index_right"].unique())

    if invert_mask:
        # Grab only the polygons that are NOT in the intersect_index.
        gpd_data_filtered = gpd_data.loc[~gpd_data.index.isin(intersect_index)]
    else:
        # Grab only the polygons that ARE in the intersect_index.
        gpd_data_filtered = gpd_data.loc[gpd_data.index.isin(intersect_index)]

    if return_inverse:
        # We need to use the indices from intersect_index to find the inverse dataset, so we
        # will just swap the '~'.

        if invert_mask:
            # Grab only the polygons that ARE in the intersect_index.
            gpd_data_inverse = gpd_data.loc[gpd_data.index.isin(
                intersect_index)]
        else:
            # Grab only the polygons that are NOT in the intersect_index.
            gpd_data_inverse = gpd_data.loc[~gpd_data.index.isin(intersect_index
                                                                )]

        return gpd_data_filtered, intersect_index, gpd_data_inverse
    else:
        return gpd_data_filtered, intersect_index

In [ ]:
def get_file_driver(file_path):
    """
    Function to optain the fiona driver to use based 
    on the file extension of a file path provided.
    
    Parameters
    ----------
    
    file_path: str 
        File path string
    
    Returns
    -------
    file_driver: str
        Fiona driver to use for the file path
        
    """

    file_extension = os.path.splitext(file_path)[-1]
    file_extension = file_extension.lower()

    if file_extension == ".geojson":
        file_driver = "GeoJSON"
    elif file_extension == ".shp":
        file_driver = "ESRI Shapefile"

    return file_driver

In [ ]:
# Generate the urban mask polygon dataset from the High Resolution Population Density Maps dataset.
# This dataset estimates the number of people living within 30-meter grid tiles in nearly every country around the world.
# https://registry.opendata.aws/dataforgood-fb-hrsl/


def generate_urban_mask_from_hrpdm(tiles, high_population_density_threshold,
                                   output_urban_mask_fp, dask_chunks):
    """
    Generate polygons for high density population areas from the High 
    Resolution Population Density Maps dataset based on the 
    `high_population_density_threshold` threshold specified. 
    
    Parameters
    ----------
    
    tiles: geopandas GeoDataFrame
        Tiles covering the area of interest. 
    dask_chunks: dict
        Chunk sizes along each dimension to use to coerce the 
        High Resolution Population Density Maps array's data 
        into a dask array. 
    high_population_density_threshold: float
        Threshold to use to generate a mask to obtain high population 
        density areas from the High Resolution Population Density Maps data.
    output_urban_mask_fp: str
        File path to write to disk the high population density polygons generated.
        
    Returns
    -------
    output_urban_mask_fp: str
        File path to where the high population density polygons have been written to disk. 
    
    """

    original_tiles_crs = tiles.crs

    output_file_driver = get_file_driver(output_urban_mask_fp)

    # Set s3 region for HRSL data access.
    os.environ['AWS_DEFAULT_REGION'] = "us-east-1"
    os.environ['AWS_S3_ENDPOINT'] = "s3.us-east-1.amazonaws.com"

    # Load the High Resolution Population Density Maps data.
    overall_population_density_fp = "s3://dataforgood-fb-data/hrsl-cogs/hrsl_general/hrsl_general-latest.vrt"
    overall_population_density = rioxarray.open_rasterio(
        overall_population_density_fp, chunks=dask_chunks).squeeze()

    # Use the defined threshold to filter the dataset to get the high population density areas.
    high_population_density = overall_population_density > high_population_density_threshold

    # Transform the tiles into the same CRS as the High Resolution Population Density Maps data.
    new_tiles_crs = overall_population_density.rio.crs
    tiles_transformed = tiles.to_crs(new_tiles_crs)
    
    counter = 0

    for tile in tiles_transformed.itertuples():
        tile_id = tile.region_code
        tile_geometry = tile.geometry
        
        minx, miny, maxx, maxy = tile_geometry.bounds

        # Subset the overall_population_density using the tile's geometry bounds.
        tile_high_population_density = high_population_density.sel(
            x=slice(minx, maxx), y=slice(maxy, miny))

        tile_high_population_density_polygons = xr_vectorize(
            tile_high_population_density,
            mask=(tile_high_population_density == True),
            crs=tile_high_population_density.geobox.crs,
            transform=tile_high_population_density.geobox.transform)

        tile_high_population_density_polygons = tile_high_population_density_polygons.to_crs(
            original_tiles_crs)

        if len(tile_high_population_density_polygons) >= 1:

            # Combine the polygons into a multi-polygon.
            merged_tile_high_population_density_polygons_geoms = shapely.ops.unary_union(
                tile_high_population_density_polygons['geometry'])

            # Turn the combined multipolygon back into a GeoDataFrame.
            merged_tile_high_population_density_polygons = gpd.GeoDataFrame(
                geometry=list(
                    merged_tile_high_population_density_polygons_geoms.geoms))

            # We need to add the crs back onto the GeoDataFrame.
            merged_tile_high_population_density_polygons.crs = original_tiles_crs

            # Save the polygons to a shapefile.
            schema = {"geometry": "Polygon"}

            if os.path.isfile(output_urban_mask_fp):
                with fiona.open(output_urban_mask_fp,
                                mode="w",
                                crs=original_tiles_crs,
                                driver=output_file_driver,
                                schema=schema) as output:
                    for row in merged_tile_high_population_density_polygons.itertuples(
                    ):
                        # Make a dictionary from the shapely Polygon object.
                        row_geom = shapely.geometry.mapping(row.geometry)
                        output.write(({'geometry': row_geom}))

            else:
                with fiona.open(output_urban_mask_fp,
                                mode="w",
                                crs=original_tiles_crs,
                                driver=output_file_driver,
                                schema=schema) as output:
                    for row in merged_tile_high_population_density_polygons.itertuples(
                    ):
                        # Make a dictionary from the shapely Polygon object.
                        row_geom = shapely.geometry.mapping(row.geometry)
                        output.write(({'geometry': row_geom}))

            counter += 1

        else:
            print(
                f'\nTile {tile_id} did not run. \n'
                f'This is probably because there are no high population density polygons present in this tile.'
            )

    # Set s3 region back for DE Africa data access.
    os.environ['AWS_DEFAULT_REGION'] = "af-south-1"
    os.environ['AWS_S3_ENDPOINT'] = "s3.af-south-1.amazonaws.com"

    if counter >= 1:
        print(
            f"\nThe high population density polygons have been written to: {output_urban_mask_fp}"
        )
        return output_urban_mask_fp
    else:
        print(
            f"\nNo high population density polygons have been found for any of the tiles."
        )
        return None

## Define Analysis Parameters

The following section walks you through the analysis parameters you will need to set for this workflow. Each section describes the parameter, how it is used, and what value was used for the DE Africa Waterbodies product.

### Set up some file names for the inputs and outputs

In [ ]:
# Create the folder to store the outputs.
output_dir = "waterbodies_outputs"
output_dir_fp = Path(output_dir)
os.makedirs(output_dir_fp, exist_ok=True)

# Set up some filenames to use.
base_filename = "SenegalBasinWaterbodies"
base_filename_fp = output_dir_fp / base_filename
              
os.makedirs(base_filename_fp, exist_ok=True)

# Putting this here to allow for testing of both methods
# to handle large polygons. 
#handle_large_polygons = 'erode-dilate-v1'
#handle_large_polygons = 'erode-dilate-v2'
handle_large_polygons = 'nothing'

# The name and filepath of the first temporary polygon dataset.
waterbodies_shapefile_temp = base_filename_fp / f"temp_{handle_large_polygons.replace('-','_')}"
# The filepath for the location of temp files during the code run.
waterbodies_shapefile_merged = base_filename_fp / f"merged_{handle_large_polygons.replace('-','_')}"
# The name and filepath of the outputs following the filtering steps
waterbodies_shapefile_filtered = base_filename_fp / f"filtered_{handle_large_polygons.replace('-','_')}"
# The name and file path of the final, completed waterbodies shapefile
waterbodies_shapefile_final = base_filename_fp / f"SenegalBasinWaterbodies_{handle_large_polygons.replace('-','_')}" #"AfricaWaterbBodies"

# File extension to use for outputs , either .shp (ESRI Shapefile) or .geojson (GeoJSON)
file_extension = ".geojson"

### Define parameters to use when loading data from the datacube

In [ ]:
dask_chunks = {'x': 3000, 'y': 3000, 'time': 1}
# Resolution of the WOfS datasets.
resolution = (-30, 30)
# CRS to work with for all files.
crs = "EPSG:6933"

# Create a datacube query.
query = dict(dask_chunks=dask_chunks, resolution=resolution, output_crs=crs)

### Set the analysis region
If you would like to perform the analysis for all of Africa, using the published WOfS All-time Summary, set `all_of_africa = True`. If you set the flag `all_of_africa` to `False`, you will need to provide either a latitude and longitude range covering the area of interest, a path to the shapefile / GeoJSON defining the area of interest, or a bounding box.

In [ ]:
all_of_africa = False

if not all_of_africa:
    #"""
    # Load a shapefile or GeoJSON file for the area of interest:
    #basin = define_area(shapefile_path="data/SenegalBasin.geojson")
    #geopolygon = geometry.Geometry(basin.features[0]["geometry"], crs="epsg:4326")
    #"""
    """
    # Use the section below if you would like to define the area of interest using a bounding box.
    # Bounding box for a section of Bukama in the Democratic Republic of Congo
    # that covers the lakes Kabwe, Kabele and Mulenda.
    bbox = (25.752395, -9.267306, 26.189124, -8.610346)
    left, bottom, right, top = bbox
    geopolygon = geometry.box(left, bottom, right, top, crs="EPSG:4326")
    """
elif all_of_africa:
    geopolygon = None

# Generate the tiles to be used in this workflow.
tiles = get_tiles(aoi_geopolygon=geopolygon,
                  all_of_africa=all_of_africa,
                  crs=crs)

# Uncomment the section below to output the tiles.
#tiles_output_fp = base_filename_fp/f"tiles{file_extension}"
#tiles.to_file(tiles_output_fp)

<a id='wetnessthreshold'></a>
### How frequently wet does a pixel need to be to be included?
The value/s set here will be the minimum frequency (as a decimal between 0 and 1) that you want water to be detected across all analysis years before it is included. 

E.g. If this was set to 0.10, any pixels that are wet *at least* 10% of the time across all valid observations will be included. If you don't want to use this filter, set this value to 0.

Following the exploration of an appropriate wetness threshold for DE Africa Waterbodies [see here]( Add-link-to-notebook-showing-threshold-sensisitivity-analysis), we choose to set two thresholds here. The code is set up to loop through both wetness thresholds, and to write out two temporary shapefiles. These two shapefiles with two separate thresholds are then used together to combine polygons from both thresholds later on in the workflow.

Polygons identified by the secondary threshold that intersect with the polygons generated by the primary threshold will be extracted, and included in the final polygon dataset. This means that the **location** of polygons is set by the primary threshold, but the **shape** of these polygons is set by the secondary threshold.

Threshold values need to be provided as a list of either one or two floating point numbers. If one number is provided, then this will be used to generate the initial polygon dataset. If two thresholds are entered, the **first number becomes the secondary threshold, and the second number becomes the primary threshold**. If more than two numbers are entered, the code will generate an error below. 

In [ ]:
# Ensure the smaller threshold is first in the list if using 2 thresholds.
secondary_threshold = 0.05
primary_threshold = 0.1
minimum_wet_thresholds = [secondary_threshold, primary_threshold]

# Test whether the wetness threshold has been correctly set.
if len(minimum_wet_thresholds) == 2:
    print(
        f'We will be running a hybrid wetness threshold. Please ensure that the major/primary threshold is \n'
        f'listed second, with the supplementary threshold entered first.  \n'
        f'**You have set {minimum_wet_thresholds[-1]} as the primary threshold, which will define the location of the waterbodies polygons** \n'
        f'**with {minimum_wet_thresholds[0]} set as the supplementary threshold, this will define the extent / shape of the waterbodies polygons**'
    )
elif len(minimum_wet_thresholds) == 1:
    print(
        f'You have not set up the hybrid threshold option. If you meant to use this option, please \n'
        f'set this option by including two wetness thresholds in the `minimum_wet_thresholds` variable above. \n'
        f'The wetness threshold we will use is {minimum_wet_thresholds}.')
else:
    raise ValueError(
        f'There is something wrong with your entered wetness threshold. Please enter a list \n'
        f'of either one or two numbers. You have entered {minimum_wet_thresholds}. \n'
        f'See above for more information')

<a id='size'></a>

### How big/small should the polygons be?
This filtering step can remove very small and/or very large waterbody polygons. The size listed here is in m<sup>2</sup>. A single pixel in Landsat data is 30 m x 30 m = 900 m<sup>2</sup>. 

**MinSize**

E.g. A minimum size of 9000 m<sup>2</sup> means that polygons need to be at least 10 pixels to be included. If you don't want to use this filter, set this value to 0.

**MaxSize**

E.g. A maximum size of 1 000 000 m<sup>2</sup> means that you only want to consider polygons less than 1 km<sup>2</sup>. If you don't want to use this filter, set this number to `math.inf`. 

*NOTE: if you are doing this analysis for all of Africa, very large polygons will be generated offshore, in the steps prior to filtering by the specified `land_sea_mask`. For this reason, we have used a `max_polygon_size` = Area of Lake Victoria (the largest lake in Africa). This will remove the huge ocean polygons, but keep large inland waterbodies that we want to map.*

In [ ]:
min_polygon_size = 4500  # 5 pixels
max_polygon_size = math.inf  #59947000000 approx area of Lake Victoria 59947 sq. km

### Filter results based on number of valid observations

The total number of valid WOfS observations for each pixel varies depending on the frequency of clouds and cloud shadow, the proximity to high slope and terrain shadow, and the seasonal change in solar angle. 

The `count_clear` parameter within the [`wofs_ls_summary_alltime`](https://explorer.digitalearth.africa/products/wofs_ls_summary_alltime) data provides a count of the number of valid observations each pixel recorded over the analysis period. We can use this parameter to mask out pixels that were infrequently observed. 
If this mask is not applied, pixels that were observed only once could be included if that observation was wet (i.e. a single wet observation means the calculation of the frequency statistic would be (1 wet observation) / (1 total observation) = 100% frequency of wet observations).

Here we set the minimum number of observations to be 128 (roughly 4 per year over our 32 year analysis). Note that this parameter does not specify the timing of these observations, but rather just the **total number of valid observations** (observed at any time of the year, in any year).

In [ ]:
this_year = datetime.datetime.now().year
start_year_wofs_dataset = 1984
min_valid_observations_yearly = 4

no_of_years = this_year - start_year_wofs_dataset

#min_valid_observations = min_valid_observations_yearly * no_of_years

min_valid_observations = 128

print(min_valid_observations)

<a id='coastline'></a>
### Read in a land/sea mask

You can choose which land/sea mask you would like to use to mask out ocean polygons, depending on how much coastal water you would like in the final product. 

We use the [Marine Regions Global Oceans and Seas v01 dataset](https://www.marineregions.org/sources.php#goas). Any polygons that intersect with this mask are filtered out, i.e. if a polygon identified within our workflow overlaps with this coastal mask by even a single pixel, it will be discarded. 

In [ ]:
land_sea_mask_fp = "data/GOaS_v1_20211214/goas_v01.shp"
#land_sea_mask_fp = "data/water-polygons-split-4326/water_polygons.shp"
land_sea_mask = gpd.read_file(land_sea_mask_fp).to_crs(crs)

<a id='rivers'></a>
### Do you want to filter out polygons that intersect with major rivers?

This filtering step is done to remove river segments from the polygon dataset. 
Set the filepath to the dataset you would wish to use in the `major_rivers_fp` variable. The dataset needs to be a vector dataset, and [able to be read in by the fiona python library](https://fiona.readthedocs.io/en/latest/fiona.html#fiona.open).

Note that we reproject this dataset to the CRS specified in the variable `crs` to match the coordinate reference system of the WOfS data we use. A list of epsg code [can be found here](https://spatialreference.org/ref/epsg/).

If you don't want to filter out polygons that intersect with rivers, set this parameter to `False`.

**Note that for the DE Africa Water Body Polygon dataset, we set this filter to False (`filter_out_rivers = False`)**

In [ ]:
filter_out_rivers = False

if filter_out_rivers:
    # Insert path to the dataset location.
    major_rivers_fp = ""
    major_rivers = gpd.GeoDataFrame.from_file(major_rivers_fp)
    major_rivers = major_rivers.to_crs(crs)

<a id='Urban'></a>

### Read in a mask for high-rise CBDs

WOfS has a known limitation, where deep shadows thrown by tall CBD buildings are misclassified as water. This results in 'waterbodies' around these misclassified shadows in capital cities. 

To address this problem, we use the High Resolution Population Density Maps dataset to define a spatial footprint for Africa's CBD areas. The theory of using this dataset is that high-rises have a high population density (population count per area). Therefore pixels in the HRPDM dataset with a higher general population density than the specified threshold are vectorized and used as our CBD filter.

If you are not using WOfS for your analysis, you may choose to set `filter_out_urban_areas = False`.

In [ ]:
filter_out_urban_areas = False #True

if filter_out_urban_areas:
    # You can load a shapefile or GeoJSON file from disk to use to filter out
    # high-rise CBDs.
    # urban_mask_fp = ""
    # urban_mask = gpd.read_file(urban_mask_fp).to_crs(crs)

    # Or you can generate the urban mask polygon dataset
    # from the High Resolution Population Density Maps dataset.
    output_fp = base_filename_fp / f'hrpdm_high_popolation_density_polygons{file_extension}'

    chunks_xy = {"x": dask_chunks["x"], "y": dask_chunks["y"]}

    urban_mask_fp = generate_urban_mask_from_hrpdm(
        tiles,
        high_population_density_threshold=10,
        output_urban_mask_fp=output_fp,
        dask_chunks=chunks_xy)

    if urban_mask_fp is not None:
        urban_mask = gpd.read_file(urban_mask_fp).to_crs(crs)

    else:
        print("\nNo high population density polygons found. \
            \n`urban_mask` set to None. \
            \nPlease use a lower threshold.")
        urban_mask = None

## Generate the first temporary polygon dataset


In [ ]:
with datacube.Datacube(app='WaterbodiesPolygons') as dc:

    for tile in tiles.itertuples():

        tile_id = tile.region_code
        tile_geopolygon = geometry.Geometry(geom=tile.geometry, crs=tiles.crs)

        # Generate waterbodies polygons for each tile.
        try:
            # Update the datacube query with the tile geometry.
            query.update(geopolygon=tile_geopolygon)

            # Load the WOfS All-Time Summary of clear and wet observations.
            wofs_alltime_summary = dc.load('wofs_ls_summary_alltime',
                                           **query).squeeze()

            # Set the no-data values to nan.
            # Masking here is done using the frequency measurement because for multiple
            # areas NaN values are present in the frequency measurement but the
            # no data value -999 is not present in the count_clear and
            # count_wet measurements.
            # Note: it seems some pixels with NaN values in the frequency measurement
            # have a value of zero in the count_clear and/or the count_wet measurements.
            wofs_alltime_summary = wofs_alltime_summary.where(
                ~np.isnan(wofs_alltime_summary.frequency))

            # Mask pixels not observed at least min_valid_observations times.
            wofs_alltime_summary_valid_clear_count = wofs_alltime_summary.count_clear >= min_valid_observations

            for threshold in minimum_wet_thresholds:
                # Mask any pixels whose frequency of water detection is less than the threshold.
                wofs_alltime_summary_valid_wetness = wofs_alltime_summary.frequency > threshold

                # Now find pixels that meet both the minimum valid observations
                # and minimum wet threshold criteria.
                wofs_alltime_summary_valid = wofs_alltime_summary_valid_wetness.where(
                    wofs_alltime_summary_valid_wetness &
                    wofs_alltime_summary_valid_clear_count)

                # Convert the raster to polygons.
                # We use a mask of '1' to only generate polygons around values of '1' (not NaNs).
                polygons_mask = wofs_alltime_summary_valid == 1

                polygons = xr_vectorize(
                    wofs_alltime_summary_valid,
                    mask=polygons_mask,
                    crs=crs,
                    transform=wofs_alltime_summary.geobox.transform)
                #polygons = polygons[polygons.attribute == 1].reset_index(drop=True)

                # Combine any overlapping polygons.
                merged_polygon_geoms = shapely.ops.unary_union(
                    polygons['geometry'])

                # Turn the combined multipolygon back into a GeoDataFrame.
                merged_polygons = gpd.GeoDataFrame(
                    geometry=list(merged_polygon_geoms.geoms))
                # We need to add the crs back onto the GeoDataFrame.
                merged_polygons.crs = crs

                # Calculate the area of each polygon again now that overlapping
                # polygons have been merged.
                merged_polygons['area'] = merged_polygons.area

                # Write the polygons to disk.
                schema = {
                    "geometry": "Polygon",
                    "properties": {
                        "area": "float",
                    }
                }

                output_fp = f'{waterbodies_shapefile_temp}_{threshold}{file_extension}'

                if os.path.isfile(output_fp):
                    with fiona.open(output_fp,
                                    mode="a",
                                    crs=crs,
                                    driver=get_file_driver(output_fp),
                                    schema=schema) as output:
                        for row in merged_polygons.itertuples():
                            row_area = row.area
                            # Make a dictionary from the shapely Polygon object.
                            row_geom = shapely.geometry.mapping(row.geometry)

                            output.write(({
                                'properties': {
                                    'area': row_area,
                                },
                                'geometry': row_geom
                            }))

                else:
                    with fiona.open(output_fp,
                                    mode="w",
                                    crs=crs,
                                    driver=get_file_driver(output_fp),
                                    schema=schema) as output:
                        for row in merged_polygons.itertuples():
                            row_area = row.area
                            # Make a dictionary from the shapely Polygon object.
                            row_geom = shapely.geometry.mapping(row.geometry)

                            output.write(({
                                'properties': {
                                    'area': row_area,
                                },
                                'geometry': row_geom
                            }))
        except:
            print(
                f'\nTile {tile_id} did not run. \n'
                f'This is probably because there are no waterbodies present in this tile.'
            )

## Merge polygons that have an edge at a tile boundary

Now that we have all of the polygons across our whole region of interest, we need to check for artifacts in the data caused by tile boundaries.

We have created a GeoDataFrame `buffered_30m_tiles`, that consists of the tile boundaries, plus a 1 pixel (30 m) buffer. This GeoDataFrame will help us to find any polygons that have a boundary at the edge of a tile. We can then find where polygons touch across this boundary, and join them up.

In [ ]:
# Add a 1 pixel (30 m) buffer to the tiles boundary.
buffered_30m_tiles = gpd.GeoDataFrame.copy(tiles, deep=True)
buffered_30m_tiles["geometry"] = buffered_30m_tiles.boundary.buffer(
    30, cap_style="flat", join_style="mitre")

In [ ]:
for threshold in minimum_wet_thresholds:
    print(f'Working on {threshold} polygons')

    # We are using the more severe wetness threshold as the main polygon dataset.
    # Note that this assumes that the thresholds have been correctly entered into the 'minimum_wet_thresholds'
    # variable, with the higher threshold listed second.
    waterbodies_polygons = gpd.read_file(
        f'{waterbodies_shapefile_temp}_{threshold}{file_extension}')

    boundary_polygons, intersect_indices, not_boundary_polygons = filter_shapefile_by_intersection(
        waterbodies_polygons,
        buffered_30m_tiles,
        invert_mask=False,
        return_inverse=True)

    # Now combine overlapping polygons in boundary_polygons.
    merged_boundary_polygons_geoms = shapely.ops.unary_union(
        boundary_polygons['geometry'])

    # `Explode` the multipolygon back out into individual polygons.
    merged_boundary_polygons = gpd.GeoDataFrame(
        crs=waterbodies_polygons.crs, geometry=[merged_boundary_polygons_geoms])
    merged_boundary_polygons = merged_boundary_polygons.explode(
        index_parts=True)

    # Then combine our merged_boundary_polygons with the not_boundary_polygons.
    all_polygons = gpd.GeoDataFrame(
        pd.concat([not_boundary_polygons, merged_boundary_polygons],
                  ignore_index=True,
                  sort=True)).set_geometry('geometry')

    # Calculate the area of each polygon in the merged polygon set.
    all_polygons["area"] = all_polygons.area

    # Check for NaNs
    all_polygons.dropna(inplace=True)

    # Export the all_polygons to disk.
    output_fp = f'{waterbodies_shapefile_merged}_{threshold}{file_extension}'

    schema = {"geometry": "Polygon", "properties": {"area": "float",}}

    with fiona.open(output_fp,
                    mode="w",
                    crs=crs,
                    driver=get_file_driver(output_fp),
                    schema=schema) as output:
        for row in all_polygons.itertuples():
            row_area = row.area
            # Make a dictionary from the shapely Polygon object.
            row_geom = shapely.geometry.mapping(row.geometry)

            output.write(({
                'properties': {
                    'area': row_area,
                },
                'geometry': row_geom
            }))

<a id='Filtering'></a>

## Filter the merged polygons by:
- **Area:**
Based on the `min_polygon_size` and `max_polygon_size` parameters set [here](#size).
- **Coastline:**
Using the `land_sea_mask` dataset loaded [here](#coastline).
- **CBD location (optional):**
Using the `urban_mask` dataset loaded [here](#Urban).
- **Wetness thresholds:**
Here we apply the hybrid threshold described [here](#wetnessthreshold)
- **Intersection with rivers (optional):**
Using the `major_rivers` dataset loaded [here](#rivers)

In [ ]:
primary_threshold_polygons = gpd.read_file(
    f'{waterbodies_shapefile_merged}_{minimum_wet_thresholds[-1]}{file_extension}'
)

primary_threshold_polygons['area'] = pd.to_numeric(
    primary_threshold_polygons.area)

# Filter out any polygons smaller than min_polygon_size, and greater than max_polygon_size.
area_filtered_primary_threshold_polygons = primary_threshold_polygons.loc[(
    (primary_threshold_polygons['area'] > min_polygon_size) &
    (primary_threshold_polygons['area'] <= max_polygon_size))]

In [ ]:
# Filtered out any ocean polygons using the land/sea mask.
inland_primary_threshold_polygons, intersect_indices = filter_shapefile_by_intersection(
    area_filtered_primary_threshold_polygons, land_sea_mask, invert_mask=True)

In [ ]:
# This was set to false in order to generate the urban mask polygon but not use it in masking.
filter_out_urban_areas = False

if filter_out_urban_areas and urban_mask is not None:
    cbd_filtered_primary_threshold_polygons, intersect_indices = filter_shapefile_by_intersection(
        inland_primary_threshold_polygons, urban_mask)
else:
    print(
        'You have chosen not to filter out waterbodies within CBDs. If you meant to use this option, please \n'
        'set `filter_out_urban_areas = True` variable above, and set the path to your urban filter shapefile'
    )
    cbd_filtered_primary_threshold_polygons = inland_primary_threshold_polygons

In [ ]:
# Check for hybrid wetness thresholds.
if len(minimum_wet_thresholds) == 2:
    # Note that this assumes that the thresholds have been correctly entered into the 'minimum_wet_thresholds'
    # variable, with the supplementary threshold listed first.
    secondary_threshold_polygons = gpd.read_file(
        f'{waterbodies_shapefile_merged}_{minimum_wet_thresholds[0]}{file_extension}'
    )
    secondary_threshold_polygons['area'] = pd.to_numeric(
        secondary_threshold_polygons.area)

    # Filter out any polygons greater than max_polygon_size.
    area_filtered_secondary_threshold_polygons = secondary_threshold_polygons.loc[
        (secondary_threshold_polygons['area'] <= max_polygon_size)]
    
    # Filtered out any ocean polygons using the land/sea mask.
    inland_secondary_threshold_polygons, intersect_indices = filter_shapefile_by_intersection(
        area_filtered_secondary_threshold_polygons, land_sea_mask, invert_mask=True)
    
    # Find the polygons identified using the secondary threshold that intersect with those identified
    # using the primary threshold.
    dont_intersect_with_primary_threshold_polygons, intersect_indices = filter_shapefile_by_intersection(
        inland_secondary_threshold_polygons,
        cbd_filtered_primary_threshold_polygons)

    do_intersect_with_primary_threshold_polygons = inland_secondary_threshold_polygons.loc[
        inland_secondary_threshold_polygons.index.isin(
            intersect_indices)]

    # Concat the two polygon datasets together.
    combined_polygons = gpd.GeoDataFrame(
        pd.concat([
            do_intersect_with_primary_threshold_polygons,
            cbd_filtered_primary_threshold_polygons
        ],
                  ignore_index=True))
    # Merge overlapping polygons.
    merged_combined_polygons_geoms = combined_polygons.unary_union
    # `Explode` the multipolygon back out into individual polygons.
    merged_combined_polygons = gpd.GeoDataFrame(
        crs=secondary_threshold_polygons.crs,
        geometry=[merged_combined_polygons_geoms])
    merged_combined_polygons = merged_combined_polygons.explode(
        index_parts=True)

else:
    print(
        'You have not set up the hybrid threshold option. If you meant to use this option, please \n'
        'set this option by including two wetness thresholds in the `minimum_wet_thresholds` variable above'
    )
    merged_combined_polygons = cbd_filtered_primary_threshold_polygons

In [ ]:
# Here is where we do the river filtering (if filter_out_rivers == True).
if filter_out_rivers:
    major_rivers_filtered_polygons, intersect_indices = filter_shapefile_by_intersection(
        merged_combined_polygons, major_rivers)
else:
    major_rivers_filtered_polygons = merged_combined_polygons

major_rivers_filtered_polygons.crs = crs

In [ ]:
# Calculate the area and perimeter of each polygon again now that overlapping polygons
# have been merged.
major_rivers_filtered_polygons['area'] = major_rivers_filtered_polygons[
    'geometry'].area
major_rivers_filtered_polygons['perimeter'] = major_rivers_filtered_polygons[
    'geometry'].length

In [ ]:
# Calculate the Polsby-Popper value (see below), and write out too.
major_rivers_filtered_polygons['pp_test'] = (
    (major_rivers_filtered_polygons['area'] * 4 * math.pi) /
    (major_rivers_filtered_polygons['perimeter']**2))


### Dividing up very large polygons

The size of polygons is determined by the contiguity of waterbody pixels through the landscape. This can result in very large polygons, e.g. where rivers are wide and unobscured by trees, or where waterbodies are connected to rivers or neighbouring waterbodies. 

We can break too large polygons into smaller, more useful polygons by applying the [Polsby-Popper test (1991)](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=2936284). The Polsby-Popper test is an assessment of the 'compactness' of a polygon. This method was originally developed to test the shape of congressional and state legislative districts, to prevent gerrymandering. 

The Polsby-Popper test examines the ratio between the area of a polygon, and the area of a circle equal to the perimeter of that polygon. The result falls between 0 and 1, with values closer to 1 being assessed as more compact.

\begin{align*}
PPtest = \frac{polygon\ area * 4\pi}{polygon\ perimeter^2}
\end{align*}

In [ ]:
## Dividing up very large polygons
# this involves a bit of manual clean up.
# The method to use to split very large polygons, by default 'nothing'
# choose 'erode-dilate-v1', 'nothing' or 'erode-dilate-v2'.
# handle_large_polygons = 'nothing'
# Threshold to use when identifying and splitting very large polygons
# using Polsby-Popper test, by default 0.005
pp_thresh = 0.005

if handle_large_polygons == 'erode-dilate-v1':
    needs_buffer = major_rivers_filtered_polygons[
        major_rivers_filtered_polygons.pp_test <= pp_thresh]
    unbuffered = needs_buffer.buffer(-50)
    unbuffered = unbuffered.explode(index_parts=True).reset_index(
        drop=True).buffer(50)
    unbuffered = gpd.GeoDataFrame(geometry=unbuffered,
                                  crs=major_rivers_filtered_polygons.crs)
    unbuffered['area'] = unbuffered.area
    unbuffered['perimeter'] = unbuffered.length
    unbuffered['pp_test'] = (unbuffered.area * 4 * math.pi /
                             unbuffered.perimeter**2)
    large_polygons_handled = pd.concat([
        major_rivers_filtered_polygons[
            major_rivers_filtered_polygons.pp_test > pp_thresh], unbuffered
    ],
                                       ignore_index=True)

if handle_large_polygons == 'erode-dilate-v2':
    splittable = major_rivers_filtered_polygons[
        major_rivers_filtered_polygons.pp_test <= pp_thresh]
    if len(splittable) >= 1:
        unbuffered = splittable.buffer(-100)
        buffered = unbuffered.buffer(125)
        subtracted = gpd.overlay(
            splittable,
            gpd.GeoDataFrame(geometry=[buffered.unary_union],
                             crs=splittable.crs),
            how='difference').explode(index_parts=True).reset_index(drop=True)
        resubtracted = gpd.overlay(
            splittable, subtracted,
            how='difference').explode(index_parts=True).reset_index(drop=True)

        # Assign each chopped-off bit of the polygon to its nearest big
        # neighbour.
        unassigned = np.ones(len(subtracted), dtype=bool)
        recombined = []
        for i, poly in resubtracted.iterrows():
            mask = (subtracted.exterior.intersects(poly.geometry.exterior) &
                    unassigned)
            neighbours = subtracted[mask]
            unassigned[mask] = False
            poly = poly.geometry.union(neighbours.unary_union)
            recombined.append(poly)

        # All remaining polygons are not part of a big polygon.
        results = pd.concat([
            gpd.GeoDataFrame(geometry=recombined,
                             crs=major_rivers_filtered_polygons.crs),
            subtracted[unassigned], major_rivers_filtered_polygons[
                major_rivers_filtered_polygons.pp_test > pp_thresh]
        ],
                            ignore_index=True)

        large_polygons_handled = results.explode(index_parts=True).reset_index(
            drop=True)

if handle_large_polygons == 'nothing':
    large_polygons_handled = major_rivers_filtered_polygons
    print('Not splitting large polygons')

In [ ]:
# Save the polygons to a shapefile
schema = {
    'geometry': 'Polygon',
    'properties': {
        'area': 'float',
        'perimeter': 'float',
        'PPtest': 'float'
    }
}

output_fp = f'{waterbodies_shapefile_filtered}{file_extension}'

with fiona.open(output_fp,
                "w",
                crs=crs,
                driver=get_file_driver(output_fp),
                schema=schema) as output:
    for row in major_rivers_filtered_polygons.itertuples():
        output.write(({
            'properties': {
                'area': row.area,
                'perimeter': row.perimeter,
                'PPtest': row.pp_test
            },
            'geometry': shapely.geometry.mapping(row.geometry)
        }))

### Final checks and recalculation of attributes

In [ ]:
filtered_polygons = gpd.read_file(
    f'{waterbodies_shapefile_filtered}{file_extension}')

In [ ]:
# Recalculate the area and perimeter of each polygon again following the manual checking
# step performed above.
filtered_polygons['area'] = filtered_polygons['geometry'].area
filtered_polygons['perimeter'] = filtered_polygons['geometry'].length
# Calculate the Polsby-Popper value (see below), and write out too.
filtered_polygons['pp_test'] = (
    (filtered_polygons['area'] * 4 * math.pi) /
    (filtered_polygons['perimeter']**2))

In [ ]:
# Keep this for now to allow for further investigation when working with the different strategies
# Remove the PPtest column, since we don't really want this as an attribute of the final shapefile.
# filtered_polygons.drop(labels='PPtest', axis=1, inplace=True)

In [ ]:
# Reapply the size filtering, just to check that all of the split and filtered waterbodies are
# still in the size range we want.
filtered_polygons = filtered_polygons.loc[(
    (filtered_polygons['area'] > min_polygon_size) &
    (filtered_polygons['area'] <= max_polygon_size))]

### Generate a unique ID for each polygon

A unique identifier is required for every polygon to allow it to be referenced. The naming convention for generating unique IDs here is the [geohash](geohash.org).

A Geohash is a geocoding system used to generate short unique identifiers based on latitude/longitude coordinates. It is a short combination of letters and numbers, with the length of the string a function of the precision of the location. The methods for generating a geohash are outlined [here - yes, the official documentation is a wikipedia article](https://en.wikipedia.org/wiki/Geohash).

Here we use the python package `python-geohash` to generate a geohash unique identifier for each polygon. We use `precision = 9` geohash characters, which represents an on the ground accuracy of <20 metres. This ensures that the precision is high enough to differentiate between waterbodies located next to each other.

In [ ]:
# We need to convert to lat/lon, in order to generate the geohash.
filtered_polygons_with_unique_ids = filtered_polygons.to_crs(epsg=4326)

# Generate a geohash for the centroid of each polygon.
filtered_polygons_with_unique_ids['UID'] = filtered_polygons_with_unique_ids.apply(lambda x: gh.encode(
    x.geometry.centroid.y, x.geometry.centroid.x, precision=9),
                                       axis=1)

# Check that our unique ID is in fact unique
assert filtered_polygons_with_unique_ids['UID'].is_unique

# Make an arbitrary numerical ID for each polygon. We will first sort the dataframe by geohash
# so that polygons close to each other are numbered similarly.
filtered_polygons_with_unique_ids_sorted = filtered_polygons_with_unique_ids.sort_values(by=['UID']).reset_index()
filtered_polygons_with_unique_ids_sorted['WB_ID'] = filtered_polygons_with_unique_ids_sorted.index

In [ ]:
# The step above creates an 'index' column, which we don't actually want, so drop it.
filtered_polygons_with_unique_ids_sorted.drop(labels='index', axis=1, inplace=True)

In [ ]:
# Write out final results to file.
final_output_fp = f"{waterbodies_shapefile_final}{file_extension}"

filtered_polygons_with_unique_ids_sorted = filtered_polygons_with_unique_ids_sorted.to_crs(crs)
filtered_polygons_with_unique_ids_sorted.to_file(final_output_fp, driver=get_file_driver(final_output_fp))